**This model is based on "Nathan, J., von Hardenberg, J. & Meron, E. Spatial instabilities untie the exclusion-principle constraint on species coexistence. Journal of Theoretical Biology 335, 198–204 (2013)."**

In [2]:
import numpy as np
import cupy as cp
from numba import jit,prange,float32,float64
from numba import cuda
import math
import matplotlib.pyplot as plt

In [3]:
def pbc(MatA):
    row,col=MatA.shape
    MatA[0,1:col-1]=MatA[row-2,1:col-1]
    MatA[row-1,1:col-1]=MatA[1,1:col-1] 
    MatA[1:row-1,0]=MatA[1:row-1,col-2] 
    MatA[1:row-1,col-1]=MatA[1:row-1,1]
    MatA[0,0]=MatA[row-2,col-2]
    MatA[0,col-1]=MatA[row-2,1]
    MatA[row-1,0]=MatA[1,col-2]
    MatA[row-1,col-1]=MatA[1,1]
    return MatA

@cuda.jit
def kernel_pbc(MatA): 
    x,y=cuda.grid(2)
    row1,col1=MatA.shape
    if (x==0) and (y>0) and (y<col1-1):
        MatA[x,y]=MatA[row1-2,y]
    elif (x==row1-1) and (y>0) and (y<col1-1):
        MatA[x,y]=MatA[1,y]
    elif (y==0) and (x>0) and (x<row1-1):
        MatA[x,y]=MatA[x,col1-2]
    elif (y==col1-1) and (x>0) and (x<row1-1):
        MatA[x,y]=MatA[x,1]
    elif (x==0) and (y==0):
        MatA[x,y]=MatA[row1-2,col1-2]
    elif (x==0) and (y==col1-1):
        MatA[x,y]=MatA[row1-2,1]
    elif (x==row1-1) and (y==0):
        MatA[x,y]=MatA[1,col1-2]
    elif (x==row1-1) and (y==col1-1):
        MatA[x,y]=MatA[1,1]
    else:
        MatA[x,y]=MatA[x,y]

@cuda.jit(device=True) #计算x方向的gradient operator
def d_dx(MatA,p_x):
    x,y=cuda.grid(2)
    return (MatA[x,y]-MatA[x-1,y])/p_x
    
@cuda.jit(device=True) #计算y方向的gradient operator
def d_dy(MatA,p_y):
    x,y=cuda.grid(2)
    return (MatA[x,y]-MatA[x,y-1])/p_y

@cuda.jit(device=True) #计算laplace operator 
def d2_dxy2(MatA,p_x,p_y):
    x,y=cuda.grid(2)
    retval=(MatA[x-1,y]-2.0*MatA[x,y]+MatA[x+1,y])/p_x/p_x+\
    (MatA[x,y-1]-2.0*MatA[x,y]+MatA[x,y+1])/p_y/p_y
    return retval
    
@cuda.jit
def kernel_ass(MatA,MatB):
    x,y=cuda.grid(2)
    MatA[x,y]=MatB[x,y]

@cuda.jit
def kernel_core(MatA,MatB,MatC,MatD,par1,par2,par3,par4,par5,par6,par7,par8,par9,par10,par11,par12,par13,MatE,MatF,MatG,MatH): 
    #MatA和MatB分别为两种植被矩阵，MatC和MatD分别为土壤和地表水分矩阵
    #MatE,MatF分别为植被的中转矩阵，MatG和MatH分别为水量的中转矩阵
    #par1,par2,par3,par4,par5分别为p1,p2,p3,p4,p5
    #par6,par7,par8,par9,par10分别为p6,p7,p8,p9,p10
    #par11,par12,par13分别为dx,dy,dt
    x,y=cuda.grid(2)
    if (x>0) and (x<MatA.shape[0]-1) and (y>0) and (y<MatA.shape[1]-1):
        Global_B1=(MatC[x,y]-1)*MatA[x,y]
        Global_B2=(par1*MatC[x,y]-par2)*MatB[x,y]
        Global_W=par4*(MatA[x,y]+par5*par6)/(MatA[x,y]+par5)*MatD[x,y]-MatC[x,y]*(par7+MatA[x,y]+MatB[x,y])
        Global_H=par9-par4*(MatA[x,y]+par5*par6)/(MatA[x,y]+par5)*MatD[x,y]
        dB1dt=Global_B1+d2_dxy2(MatA,par11,par12)
        dB2dt=Global_B2+par3*d2_dxy2(MatB,par11,par12)
        dWdt=Global_W+par8*d2_dxy2(MatC,par11,par12)
        dHdt=Global_H+par10*d2_dxy2(MatD,par11,par12)
        MatE[x,y]=MatA[x,y]+dB1dt*par13
        MatF[x,y]=MatB[x,y]+dB2dt*par13
        MatG[x,y]=MatC[x,y]+dWdt*par13
        MatH[x,y]=MatD[x,y]+dHdt*par13
    else:
        MatE[x,y]=MatA[x,y]
        MatF[x,y]=MatB[x,y]
        MatG[x,y]=MatC[x,y]
        MatH[x,y]=MatD[x,y]

In [ ]:
# Basic settings
DT=np.float32
T=np.int(100) #演化时间
L=np.int(4096) #空间尺度
M=L  
N=L
B1_d=cp.zeros(shape=(M,N),dtype=DT) #第一种植被生物量矩阵
B1_temp_d=cp.zeros(shape=(M,N),dtype=DT)
B2_d=cp.zeros(shape=(M,N),dtype=DT) #第二种植被生物量矩阵
B2_temp_d=cp.zeros(shape=(M,N),dtype=DT)
W_d=cp.zeros(shape=(M,N),dtype=DT) #水量矩阵
W_temp_d=cp.zeros(shape=(M,N),dtype=DT)
H_d=cp.empty(shape=(M,N),dtype=DT)
H_temp_d=cp.zeros(shape=(M,N),dtype=DT)

#Model parameters, use the parameters in Fig. 2 of the paper
p1=np.float32(1) #c2
p2=np.float32(1) #miu2
p3=np.float32(1) #d2
p4=np.float32(40) #alfa
p5=np.float32(0.1) #q
p6=np.float32(0.1) #f
p7=np.float32(1) #n
p8=np.float32(100) #dw
p9=np.float32(1) #p
p10=np.float32(10000) #dh
dx=np.float32(1) #grid size of x direction
dy=np.float32(1) #grid size of y direction
dt=np.float32(0.1) #time interval

#GPU settings
threadsperblock=(32,32)
blockspergrid_x=math.ceil(M/threadsperblock[0])
blockspergrid_y=math.ceil(N/threadsperblock[1])
blockspergrid=(blockspergrid_x,blockspergrid_y)

#Initial conditions
B1_d=(cp.where(cp.random.rand(M,N)<0.5,1,0)*0.25+1).astype(DT)
B2_d=(cp.where(cp.random.rand(M,N)<0.5,1,0)*0.25+1).astype(DT)
W_d=(W+1).astype(DT)
H_d=(H+1).astype(DT)
kernel_pbc[blockspergrid,threadsperblock](B1_d)
kernel_pbc[blockspergrid,threadsperblock](B2_d)
kernel_pbc[blockspergrid,threadsperblock](W_d)
kernel_pbc[blockspergrid,threadsperblock](H_d)

#Model evolution
for k in range(T):
    kernel_core[blockspergrid,threadsperblock](B1_d,B2_d,W_d,H_d,p1,p2,p3,p4,p5,p6,p7,p8,p9,p10,dx,dy,dt,B1_temp_d,B2_temp_d,W_temp_d,H_temp_d)
    kernel_ass[blockspergrid,threadsperblock](B1_d,B1_temp_d)
    kernel_ass[blockspergrid,threadsperblock](B2_d,B2_temp_d)
    kernel_ass[blockspergrid,threadsperblock](W_d,W_temp_d)
    kernel_ass[blockspergrid,threadsperblock](H_d,H_temp_d)
    kernel_pbc[blockspergrid,threadsperblock](B1_d)
    kernel_pbc[blockspergrid,threadsperblock](B2_d)
    kernel_pbc[blockspergrid,threadsperblock](W_d)
    kernel_pbc[blockspergrid,threadsperblock](H_d)
#    plt.imshow(VG)
#    plt.show()